# Задача #
посчитать метрики:
●	Delivery rate
●	Open rate
●	Click to Open rate
●	Unsubscribe rate

●	Выявить лучшую тему

In [1]:
### Импорт библиотек и данных

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu, ttest_ind
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [2]:

mailfit_df = pd.read_excel('Data для тестового.xlsx', sheet_name = None)
df = mailfit_df['Data']
df.head(5)

,Название рассылки,Название кампании,Направление,Месяц,Дата,Год,Номер недели,День недели,День недели2,Время,...,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,UTM Метка,Пользователей на сайте,Воронка продаж. Шаг 1,Воронка продаж. Шаг 2,Воронка продаж. Шаг 3
0,Название рассылки 1,Название кампании 1,Email,Октябрь,2021-10-27,2021,43,3,03-среда,19:24:00,...,741749.55,148349.910,17801.989200,39039.45,7417.4955,Метка 1,16377.830064,6336.582452,6209.850803,5154.176166
1,Название рассылки 10,Название кампании 10,Email,Ноябрь,2021-11-05,2021,45,5,05-пятница,12:02:00,...,683401.50,123012.270,11071.104300,35968.50,6834.0150,Метка 10,10296.126999,3558.341491,3095.757097,2538.520820
2,Название рассылки 100,Название кампании 100,Email,Апрель,2022-04-11,2022,15,1,01-понедельник,16:26:00,...,1141344.25,182615.080,15339.666720,60070.75,11413.4425,Метка 100,13959.096715,3369.725947,2864.267055,2262.770973
3,Название рассылки 101,Название кампании 101,Email,Апрель,2022-04-12,2022,15,2,02-вторник,16:26:00,...,1324135.65,264827.130,10328.258070,69691.35,13241.3565,Метка 101,8985.584521,5116.391826,4277.303567,3207.977675
4,Название рассылки 102,Название кампании 102,Email,Апрель,2022-04-13,2022,15,3,03-среда,16:26:00,...,1212979.95,218336.391,15720.220152,63841.05,12129.7995,Метка 102,10846.951905,2816.953410,2287.366169,1669.777303


In [3]:
### Есть-ли дубли 

df.duplicated().sum()

np.int64(0)

In [4]:
### Есть-ли нулы

df.isnull().sum()

Название рассылки         0
Название кампании         0
Направление               0
Месяц                     0
Дата                      0
Год                       0
Номер недели              0
День недели               0
День недели2              0
Время                     0
Веб-версия                0
Тема письма               0
Сегмент                   0
Отправлено                0
Доставлено                0
Открытия                  0
Клики                     0
Баунсы (Все ошибки)       0
Отписки                   0
UTM Метка                 0
Пользователей на сайте    0
Воронка продаж. Шаг 1     0
Воронка продаж. Шаг 2     0
Воронка продаж. Шаг 3     0
dtype: int64

In [5]:
### Либо можно вывести только info - смотрим и форматы и нулы

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Название рассылки       218 non-null    object        
 1   Название кампании       218 non-null    object        
 2   Направление             218 non-null    object        
 3   Месяц                   218 non-null    object        
 4   Дата                    218 non-null    datetime64[ns]
 5   Год                     218 non-null    int64         
 6   Номер недели            218 non-null    int64         
 7   День недели             218 non-null    int64         
 8   День недели2            218 non-null    object        
 9   Время                   218 non-null    object        
 10  Веб-версия              218 non-null    object        
 11  Тема письма             218 non-null    object        
 12  Сегмент                 218 non-null    object    

In [6]:
### Посчитаем все метрики

df['delivery_rate'] = df['Доставлено'].replace(0, pd.NA) / df['Отправлено'].replace(0, pd.NA) * 100
df['open_rate'] = df['Открытия'].replace(0, pd.NA) / df['Доставлено'].replace(0, pd.NA) * 100
df['ctor'] = df['Клики'].replace(0, pd.NA) / df['Открытия'].replace(0, pd.NA) * 100
df['ur'] = df['Отписки'].replace(0, pd.NA) / df['Доставлено'].replace(0, pd.NA) * 100
df.head(5)

,Название рассылки,Название кампании,Направление,Месяц,Дата,Год,Номер недели,День недели,День недели2,Время,...,Отписки,UTM Метка,Пользователей на сайте,Воронка продаж. Шаг 1,Воронка продаж. Шаг 2,Воронка продаж. Шаг 3,delivery_rate,open_rate,ctor,ur
0,Название рассылки 1,Название кампании 1,Email,Октябрь,2021-10-27,2021,43,3,03-среда,19:24:00,...,7417.4955,Метка 1,16377.830064,6336.582452,6209.850803,5154.176166,95.0,20.0,12.0,1.0
1,Название рассылки 10,Название кампании 10,Email,Ноябрь,2021-11-05,2021,45,5,05-пятница,12:02:00,...,6834.0150,Метка 10,10296.126999,3558.341491,3095.757097,2538.520820,95.0,18.0,9.0,1.0
2,Название рассылки 100,Название кампании 100,Email,Апрель,2022-04-11,2022,15,1,01-понедельник,16:26:00,...,11413.4425,Метка 100,13959.096715,3369.725947,2864.267055,2262.770973,95.0,16.0,8.4,1.0
3,Название рассылки 101,Название кампании 101,Email,Апрель,2022-04-12,2022,15,2,02-вторник,16:26:00,...,13241.3565,Метка 101,8985.584521,5116.391826,4277.303567,3207.977675,95.0,20.0,3.9,1.0
4,Название рассылки 102,Название кампании 102,Email,Апрель,2022-04-13,2022,15,3,03-среда,16:26:00,...,12129.7995,Метка 102,10846.951905,2816.953410,2287.366169,1669.777303,95.0,18.0,7.2,1.0


In [7]:
### Посмотрим на Delivery rate, остальные можно представить так же

deliv_rate = pd.pivot_table(df, index = 'Название рассылки', columns = 'Дата', values = 'delivery_rate', aggfunc = 'mean')
deliv_rate.head(5)

Дата,2021-04-15,2021-04-21,2021-04-22,2021-04-23,2021-04-30,2021-05-05,2021-05-06,2021-05-11,2021-05-26,2021-06-03,...,2022-04-12,2022-04-13,2022-04-14,2022-04-15,2022-04-22,2022-04-26,2022-05-05,2022-05-12,2022-05-16,2022-05-20
Название рассылки,,,,,,,,,,,,,,,,,,,,,
Название рассылки 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Название рассылки 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Название рассылки 100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Название рассылки 101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Название рассылки 102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
### Найдем лучшую тему(название рассылки, название компании), допустим по значению - лучшая конверсия минус отписка

df['conv_deliv_minus_unsubsc_to_clic'] = df['Клики'].replace(0, pd.NA) / (df['Доставлено'].replace(0, pd.NA) - df['Отписки'].replace(0, pd.NA)) * 100
selected = df[['Название рассылки', 'ur', 'conv_deliv_minus_unsubsc_to_clic']].sort_values(by='conv_deliv_minus_unsubsc_to_clic', ascending=False)
selected.head(10)

,Название рассылки,ur,conv_deliv_minus_unsubsc_to_clic
135,Название рассылки 24,13.00,2.482759
145,Название рассылки 33,13.00,2.482759
155,Название рассылки 42,11.00,2.426966
0,Название рассылки 1,1.00,2.424242
55,Название рассылки 149,0.59,2.172820
65,Название рассылки 158,0.59,2.172820
210,Название рассылки 92,0.90,2.110595
185,Название рассылки 7,0.90,2.110595
200,Название рассылки 83,0.90,2.110595
195,Название рассылки 79,0.90,2.110595


## Не углубляясь, на данном этапе замечаем, что у рассылок 24, 33, 42 высокая конверсия из (доставленных минус отписки) в клики ##
## Но нам важно учитывать и отписки, а вот с их учетом замечаем "лидерство" рассылки 1 ##

In [9]:
### А теперь посмотрим, есть-ли статистически значимая разница между полученными результатами, т.е. действительно-ли "Название рассылки 24" лучшая

# Рассылка 24
n1 = df[df['Название рассылки'] == 'Название рассылки 24']['Доставлено'].sum()  # доставлено
unsub1 = df[df['Название рассылки'] == 'Название рассылки 24']['Отписки'].sum() # отписались
k1 = df[df['Название рассылки'] == 'Название рассылки 24']['Клики'].sum()     # кликов

# Рассылка 1
n2 = df[df['Название рассылки'] == 'Название рассылки 1']['Доставлено'].sum()
unsub2 = df[df['Название рассылки'] == 'Название рассылки 1']['Отписки'].sum()
k2 = df[df['Название рассылки'] == 'Название рассылки 1']['Клики'].sum()

eff_n1 = n1 - unsub1
eff_n2 = n2 - unsub2

conv1 = k1 / eff_n1
conv2 = k2 / eff_n2

count = [k1, k2]
nobs = [eff_n1, eff_n2]

z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

# Доверительные интервалы (метод Уилсона)
ci1 = proportion_confint(k1, eff_n1, alpha=0.05, method='wilson')
ci2 = proportion_confint(k2, eff_n2, alpha=0.05, method='wilson')


print(f"p-value = {p_value:.4f}")
if p_value < 0.05:
    print("  → Разница статистически значима!")
else:
    print("  → Разница не является статистически значимой.")

print(f"\nКонверсия рассылки 24: {conv1:.2%} (95% ДИ: {ci1[0]:.2%}–{ci1[1]:.2%})")
print(f"Конверсия рассылки 1:  {conv2:.2%} (95% ДИ: {ci2[0]:.2%}–{ci2[1]:.2%})")

p-value = 0.0164
  → Разница статистически значима!

Конверсия рассылки 24: 2.48% (95% ДИ: 2.45%–2.52%)
Конверсия рассылки 1:  2.42% (95% ДИ: 2.39%–2.46%)


## На данном этапе замечаем, что если не смотреть (но учитывать в расчетах) высокую отписку, рассылка 24 лучшая ##

In [10]:
### С учетом выводов из предыдущего окна следует сравнить "Название рассылки 24" и "Название рассылки 33"

# Рассылка 24
n11 = df[df['Название рассылки'] == 'Название рассылки 24']['Доставлено'].sum()  # доставлено
unsub11 = df[df['Название рассылки'] == 'Название рассылки 24']['Отписки'].sum() # отписались
k11 = df[df['Название рассылки'] == 'Название рассылки 24']['Клики'].sum()     # кликов

# Рассылка 1
n22 = df[df['Название рассылки'] == 'Название рассылки 33']['Доставлено'].sum()
unsub22 = df[df['Название рассылки'] == 'Название рассылки 33']['Отписки'].sum()
k22 = df[df['Название рассылки'] == 'Название рассылки 33']['Клики'].sum()

eff_n11 = n11 - unsub11
eff_n22 = n22 - unsub22

conv11 = k11 / eff_n11
conv22 = k22 / eff_n22

count = [k11, k22]
nobs = [eff_n11, eff_n22]

z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

# Доверительные интервалы (метод Уилсона)
ci11 = proportion_confint(k11, eff_n11, alpha=0.05, method='wilson')
ci22 = proportion_confint(k22, eff_n22, alpha=0.05, method='wilson')


print(f"p-value = {p_value:.4f}")
if p_value < 0.05:
    print("  → Разница статистически значима!")
else:
    print("  → Разница не является статистически значимой.")

print(f"\nКонверсия рассылки 24: {conv1:.2%} (95% ДИ: {ci1[0]:.2%}–{ci1[1]:.2%})")
print(f"Конверсия рассылки 33:  {conv2:.2%} (95% ДИ: {ci2[0]:.2%}–{ci2[1]:.2%})")

p-value = 1.0000
  → Разница не является статистически значимой.

Конверсия рассылки 24: 2.48% (95% ДИ: 2.45%–2.52%)
Конверсия рассылки 33:  2.42% (95% ДИ: 2.39%–2.46%)


## Рассылка 24 по результативности равна рассылке 33 ##

# ВЫВОД: по совокупности высокой конверсии в клик и минимальной отписки лучшей является Рассылка 1 #